### Node.js Installation
We require the ability to run JavaScript from this notebook.

**Install Node.js dependencies inside of this JupyterLab Docker container**

In [1]:
!mamba install --yes make cxx-compiler


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.18.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['make', 'cxx-compiler']

conda-forge/linux-64     [<=>                 ] (00m:00s) 
conda-forge/li

**Install Node.js Jupyter kernel inside of this JupyterLab Docker container**

In [2]:
!npm install -g ijavascript
!ijsinstall

npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
/opt/conda/bin/ijsconsole -> /opt/conda/lib/node_modules/ijavascript/bin/ijsconsole.js:refresh-package-jsmK
/opt/conda/bin/ijs -> /opt/conda/lib/node_modules/ijavascript/bin/ijavascript.js
/opt/conda/bin/ijskernel -> /opt/conda/lib/node_modules/ijavascript/lib/kernel.js
/opt/conda/bin/ijsinstall -> /opt/conda/lib/node_modules/ijavascript/bin/ijsinstall.js
/opt/conda/bin/ijsnotebook -> /opt/conda/lib/node_modules/ijavascript/bin/ijsnotebook.js

> zeromq@5.2.8 install /opt/conda/lib/node_modules/ijavascript/node_modules/zeromq
> node-gyp-build || npm run build:libzmq

+ ijavascript@5.2.1
added 8 packages from 31 contributors in 7.823s


**Create Node.js project in the current directory and install Fabric Gateway dependencies**

In [3]:
!npm init -y
!npm install fabric-ca-client@2.2.11 fabric-network@2.2.11 js-yaml@3.14.1

Wrote to /home/jovyan/work/notebook/package.json:

{
  "name": "notebook",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}




   ╭───────────────────────────────────────────────────────────────╮
   │                                                               │
   │       New major version of npm available! 6.9.0 → 8.5.2       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v8.5.2   │
   │               Run npm install -g npm to update!               │
   │                                                               │
   ╰───────────────────────────────────────────────────────────────╯

npm WARN deprecated querystring@0.2.0: The querystring API is considered Legacy. new code should use the URLSearchParams API instead.
        ......] | build:fabric-common: sill linkStuff fabric-common@2.2.11K0m
> pkcs11js@1.3.0 install /h

### User Registration

**The CA admin first registers a user (on the Docker host):** \
docker exec -it ca-client.fabric.localhost ./registerUser.js --profile ../connection-profiles/connection-profile-orga.yaml --org orgA --user janssen

### Switch to JavaScript (Node.js) kernel!

**Import modules**

In [1]:
const { Wallets, x509Identity, Gateway } = require('fabric-network');
const FabricCAServices = require('fabric-ca-client');
const yaml = require('js-yaml');
const fs = require('fs');
const path = require('path');

**Configure the user (using the credentials provided by the organization's CA)**

In [2]:
const org = 'orgA';
const user = 'janssen';
const secret = 'UHSIzQbKkyzp';
const connectionProfile = yaml.safeLoad(fs.readFileSync(path.resolve(__dirname, '../connection-profiles/connection-profile-orga.yaml'),'utf8'));

### CA Enrollment

In [ ]:
async function enroll() {
  // Set up the CA connection
  var certAuth = connectionProfile.organizations[org].certificateAuthorities[0];
  const caInfo = connectionProfile.certificateAuthorities[certAuth];
  const caTLSCACerts = caInfo.tlsCACerts.pem;
  const ca = new FabricCAServices(caInfo.url, { trustedRoots: caTLSCACerts, verify: false }, caInfo.caName);
    
  // Enroll the user
  const enrollment = await ca.enroll({ enrollmentID: user, enrollmentSecret: secret });
    
  // Save the identity in the wallet
  const mspId = connectionProfile.organizations[org].mspid;
  const identity = {
    credentials: {
      certificate: enrollment.certificate,
      privateKey: enrollment.key.toBytes(),
    },
    mspId: mspId,
    type: 'X.509',
  };
  const wallet = await Wallets.newFileSystemWallet('./wallet');
  wallet.put(user, identity);
    
  console.log('Enrollment complete!');
}
enroll();

### Gateway / Scenarios

In [4]:
async function gateway() {
  const wallet = await Wallets.newFileSystemWallet('./wallet')
  const gateway = new Gateway();
  try { 
    await gateway.connect(connectionProfile, {
      wallet,
      identity: user,
      discovery: {
        enabled: true,
        asLocalhost: false
      }
    });
    const network = await gateway.getNetwork('consortium-chain');
    const contract = network.getContract('consortium-cc-ipfs');
    
    // Submit transactions for the smart contract
    const submitResult = await contract.submitTransaction('put', 'test', 'os3');

    console.log('Transaction complete!');
  } catch(error) {
    console.log(error);
  } finally {
    gateway.disconnect();
  }
}
gateway();

Promise { <pending> }

Transaction complete!
